In [1]:
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import FileSearchTool, MessageAttachment, FilePurpose
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

load_dotenv()

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.getenv('PROJECT_CONNECTION_STRING')
)

# Azure AI Search

https://learn.microsoft.com/en-us/python/api/overview/azure/search-documents-readme?view=azure-python#key-concepts

https://learn.microsoft.com/en-us/azure/search/samples-python

In [4]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
import os 

service_endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
index_name = os.environ["AZURE_SEARCH_INDEX_NAME"]
key = os.environ["AZURE_SEARCH_API_KEY"]

search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))
search_client.get_document_count()

83

## Query

In [5]:
results = search_client.search(search_text="RL generalizes")

res = []
for result in results:
    res.append(result)

res[0]

{'metadata_security_id': [],
 'id': 'b2ec5d01c68b_RQBwADIANgB0AEcASgBHAFEATwBpAGsAQwBHAEoAOABCAFMAegBCAG0AaABPAGIATgBnAGwANwBYAHEAbQBwAEIATQBpADAAbQBsAFUAbgBFAFYAUgBtAHYAcAB4AGsANQBWAHYAegAxAHcAQwB0AFYALQBDADAAVQBtADkARgBJAGMAVgBHAFcASABvAFMAMQBxAFIAdABvAHgARQBxAGcAdQBOAHIANgA0AEEAMgA1_chunks_3',
 'metadata_storage_last_modified': '2025-04-01T19:53:22Z',
 'metadata_storage_name': 'SFT Memorizes, RL Generalizes.pdf',
 'page': 6,
 'summary': '',
 'url': 'https://strag05malxg62kzvgk.blob.core.windows.net/documents/SFT%20Memorizes,%20RL%20Generalizes.pdf',
 'relatedImages': [],
 'filepath': 'SFT%20Memorizes,%20RL%20Generalizes.pdf',
 'imageCaptions': '',
 'metadata_storage_path': 'https://strag05malxg62kzvgk.blob.core.windows.net/documents/SFT%20Memorizes,%20RL%20Generalizes.pdf',
 'relatedFiles': [],
 'title': 'Sft%20 Memorizes,%20 Rl%20 Generalizes',
 'parent_id': 'RQBwADIANgB0AEcASgBHAFEATwBpAGsAQwBHAEoAOABCAFMAegBCAG0AaABPAGIATgBnAGwANwBYAHEAbQBwAEIATQBpADAAbQBsAFUAbgBFAFYAUgBtAHYAcAB4

## Search with chat model

https://learn.microsoft.com/es-es/azure/search/tutorial-rag-build-solution-query

In [29]:
# Import libraries
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
import os


# Initialize environment variables
API_KEY = os.getenv("api_key")
PROJECT_CONNECTION_STRING = os.getenv("PROJECT_CONNECTION_STRING")
BING_CONNECTION_NAME = os.getenv("BING_CONNECTION_NAME")
# MODEL_DEPLOYMENT_NAME = os.getenv("MODEL_DEPLOYMENT_NAME")
MODEL_API_VERSION = os.getenv("MODEL_API_VERSION")
AZURE_ENDPOINT = os.getenv("AZURE_ENDPOINT")
BING_MODEL_DEPLOYMENT_NAME = os.getenv("BING_MODEL_DEPLOYMENT_NAME")
AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("AGENT_MODEL_DEPLOYMENT_NAME")


agent_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=AGENT_MODEL_DEPLOYMENT_NAME,
    model=AGENT_MODEL_DEPLOYMENT_NAME,
    api_version=MODEL_API_VERSION,
    azure_endpoint=AZURE_ENDPOINT,
    api_key=API_KEY,
)


# Provide instructions to the model
GROUNDED_PROMPT="""
You are an AI assistant that helps users learn from the information found in the source material.
Answer the query using only the sources provided below.
Use bullets if the answer has multiple points.
If the answer is longer than 3 sentences, provide a summary.
Answer ONLY with the facts listed in the list of sources below. Cite your source when you answer the question
If there isn't enough information below, say you don't know.
Do not generate answers that don't use the sources below.
Query: {query}
Sources:\n{sources}

"""

from azure.search.documents.models import VectorizableTextQuery


# Provide the search query. 
# It's hybrid: a keyword search on "query", with text-to-vector conversion for "vector_query".
# The vector query finds 50 nearest neighbor matches in the search index
query="What does it mean that RL generalizes?"
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="content")

# Set up the search results and the chat thread.
# Retrieve the selected fields from the search index related to the question.
# Search results are limited to the top 5 matches. Limiting top can help you stay under LLM quotas.
search_results = search_client.search(
    search_text=query,
    vector_queries=[vector_query],
    select=["content", "title", "category", "url", "source"],
    top=5,
)

# Display the search results
for result in search_results:
    print(result)

HttpResponseError: (OperationNotAllowed) The field 'content' in the vector field list is not a vector field.
Parameter name: vector.fields
Code: OperationNotAllowed
Message: The field 'content' in the vector field list is not a vector field.
Parameter name: vector.fields
Exception Details:	(FieldNotSearchable) The field 'content' in the vector field list is not a vector field.
	Code: FieldNotSearchable
	Message: The field 'content' in the vector field list is not a vector field.

In [26]:
for result in search_results:
    print(result)


TypeError: Session.request() got an unexpected keyword argument 'vector'

In [21]:
from azure.search.documents.models import VectorizableTextQuery


# Provide the search query. 
# It's hybrid: a keyword search on "query", with text-to-vector conversion for "vector_query".
# The vector query finds 50 nearest neighbor matches in the search index
query="What does it mean that RL generalizes?"
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="content")

# Set up the search results and the chat thread.
# Retrieve the selected fields from the search index related to the question.
# Search results are limited to the top 5 matches. Limiting top can help you stay under LLM quotas.
search_results = search_client.search(
    search_text=query,
    vector_queries= [vector_query],
    select=["content", "title", "category", "url", "source", "contentVector", "captionVector"],
    top=5,
)

search_results

In [22]:
# Set up the search results and the chat thread.
# Retrieve the selected fields from the search index related to the question.
# Search results are limited to the top 5 matches. Limiting top can help you stay under LLM quotas.
search_results = search_client.search(
    search_text=query,
    vector_queries= [vector_query],
    select=["content", "title", "category", "url", "source", "contentVector", "captionVector"],
    top=5,
)

search_results

<iterator object azure.core.paging.ItemPaged at 0x241c6694dd0>

In [23]:
for result in search_results:
    print(result)


HttpResponseError: (InvalidRequestParameter) At least one vector field needs to be selected explicitly using the 'vector.fields' parameter.
Parameter name: vector.fields
Code: InvalidRequestParameter
Message: At least one vector field needs to be selected explicitly using the 'vector.fields' parameter.
Parameter name: vector.fields
Exception Details:	(InvalidVectorQuery) At least one vector field needs to be selected explicitly using the 'vector.fields' parameter.
	Code: InvalidVectorQuery
	Message: At least one vector field needs to be selected explicitly using the 'vector.fields' parameter.

In [12]:
# Newlines could be in the OCR'd content or in PDFs, as is the case for the sample PDFs used for this tutorial.
# Use a unique separator to make the sources distinct. 
# We chose repeated equal signs (=) followed by a newline because it's unlikely the source documents contain this sequence.
sources_formatted = "=================\n".join([f'TITLE: {document["title"]}, CONTENT: {document["chunk"]}, LOCATIONS: {document["locations"]}' for document in search_results])


HttpResponseError: (InvalidRequestParameter) Unknown field 'text_vector' in vector field list.
Code: InvalidRequestParameter
Message: Unknown field 'text_vector' in vector field list.
Exception Details:	(UnknownField) Unknown field 'text_vector' in vector field list.
	Code: UnknownField
	Message: Unknown field 'text_vector' in vector field list.

## Add document to index

Para esto hay que guardarlo en el blob storage

In [ ]:
import os
from pathlib import Path
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchableField,
    SimpleField,
    SearchIndex,
    SearchFieldDataType
)

# Path to your local file
file_path = "path/to/your/local/file.pdf"  # Update with your actual file path

# First, you need to upload the file to Azure Blob Storage
# This requires a separate Azure Blob Storage client setup
from azure.storage.blob import BlobServiceClient

# You would need your blob storage connection string
blob_connection_string = "your_blob_connection_string"  # Replace with actual connection string
blob_service_client = BlobServiceClient.from_connection_string(blob_connection_string)

# Create a container if it doesn't exist
container_name = "documents"
container_client = blob_service_client.get_container_client(container_name)
if not container_client.exists():
    container_client.create_container()

# Upload the file to blob storage
blob_name = Path(file_path).name
blob_client = container_client.get_blob_client(blob_name)

with open(file_path, "rb") as data:
    blob_client.upload_blob(data, overwrite=True)

print(f"File uploaded to blob storage: {blob_name}")

# Now the file is in blob storage and can be indexed by Azure AI Search
# You would typically use an indexer to pull content from blob storage into your search index
# This requires setting up a data source and indexer in your search service

print("File uploaded to blob storage. Use Azure Portal to create an indexer to index this file.")